<a href="https://colab.research.google.com/github/AlejandraMatajira/Mi-repositorio-/blob/main/Simulacion_MonteCarlo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
import seaborn as sns

class MonteCarloSimulation:
    def __init__(self, probabilidades_mensuales=None):

        # Probabilidades especÃ­ficas por mes (formato: {'mes': {'devolucion': prob, 'venta': prob}})
        self.probabilidades_mensuales = probabilidades_mensuales or {}


    def generar_volumen_venta(self):
        # 1.72e+03 + WEIB(1.82e+04, 0.448)
        return 1.72e+03 + np.random.weibull(0.448) * 1.82e+04

    def generar_precio_venta(self):
        # 1.36e+04 + GAMM(1.89e+04, 0.68)
        return 1.36e+04 + np.random.gamma(shape=1.89e+04, scale=0.68)

    def generar_costo_mp_venta(self):
        # 2.59e+03 + 5.27e+03 * BETA(0.375, 0.668)
        return 2.59e+03 + 5.27e+03 * np.random.beta(0.375, 0.668)

    def generar_otros_costos_venta(self):
        # 6.25e+03 + GAMM(7.71e+03, 0.424)
        return 6.25e+03 + np.random.gamma(shape=7.71e+03, scale=0.424)

    # ------- DEVOLUCIONES PARA LUKER -------
    def generar_volumen_devolucion(self):
        # TRIA(-1.05e+03, -106, -0.999)
        return np.random.triangular(-1.05e+03, -106, -0.999)

    def generar_precio_devolucion(self):
        # 251 + EXPO(3.28e+04)
        return 251 + np.random.exponential(3.28e+04)

    def generar_costo_mp_devolucion(self):
        # 251 + EXPO(3.28e+04)
        return 251 + np.random.exponential(3.28e+04)

    def generar_otros_costos_devolucion(self):
        # -0.001 + WEIB(5.55e+03, 0.554)
        return -0.001 + np.random.weibull(0.554) * 5.55e+03
    def get_probabilidades_mes(self, mes):
        """Obtiene las probabilidades especÃ­ficas para un mes dado"""
        if mes in self.probabilidades_mensuales:
            return (
                self.probabilidades_mensuales[mes].get('devolucion'),
                self.probabilidades_mensuales[mes].get('venta')
            )

    def simular_mes(self, mes_nombre, n_simulaciones=10000):
        """Simula un mes completo con n_simulaciones iteraciones"""
        prob_devolucion, prob_venta = self.get_probabilidades_mes(mes_nombre)
        utilidades_total = []
        utilidades_devoluciones = []
        utilidades_ventas = []

        conteo_eventos = {
            'solo_devolucion': 0,
            'solo_venta': 0,
            'ambos_eventos': 0,
            'ningun_evento': 0
        }

        for _ in range(n_simulaciones):
            utilidad_devolucion = 0
            utilidad_venta = 0
            ocurre_devolucion = False
            ocurre_venta = False

            # Evaluar si ocurre devoluciones (probabilidad especÃ­fica del mes)
            if np.random.random() < prob_devolucion:
                ocurre_devolucion = True
                vol_dev = self.generar_volumen_devolucion()
                precio_dev = self.generar_precio_devolucion()
                costo_mp_dev = self.generar_costo_mp_devolucion()
                otros_costos_dev = self.generar_otros_costos_devolucion()

                # Utilidad de devoluciones (negativa)
                utilidad_devolucion = vol_dev * (precio_dev - costo_mp_dev - otros_costos_dev)

            # Evaluar si ocurre venta (probabilidad especÃ­fica del mes, independiente de devoluciÃ³n)
            if np.random.random() < prob_venta:
                ocurre_venta = True
                vol_venta = self.generar_volumen_venta()
                precio_venta = self.generar_precio_venta()
                costo_mp_venta = self.generar_costo_mp_venta()
                otros_costos_venta = self.generar_otros_costos_venta()

                # Utilidad de venta (positiva)
                utilidad_venta = vol_venta * (precio_venta - costo_mp_venta - otros_costos_venta)

            # Contabilizar tipos de eventos para analisis
            if ocurre_devolucion and ocurre_venta:
                conteo_eventos['ambos_eventos'] += 1
            elif ocurre_devolucion and not ocurre_venta:
                conteo_eventos['solo_devolucion'] += 1
            elif not ocurre_devolucion and ocurre_venta:
                conteo_eventos['solo_venta'] += 1
            else:  # No ocurre ningÃºn evento
                conteo_eventos['ningun_evento'] += 1

            # Guardar utilidades por separado
            utilidades_devoluciones.append(utilidad_devolucion)
            utilidades_ventas.append(utilidad_venta)
            utilidades_total.append(utilidad_devolucion + utilidad_venta)

        return {
            'total': np.array(utilidades_total),
            'devoluciones': np.array(utilidades_devoluciones),
            'ventas': np.array(utilidades_ventas),
            'conteo_eventos': conteo_eventos
        }

    def calcular_estadisticas(self, utilidades, nombre_evento):
        """Calcula estadísticas robustas para un conjunto de utilidades"""
        utilidades = np.array(utilidades)
        utilidades_filtradas = utilidades[utilidades != 0]

        # Probabilidades sobre el total original
        prob_utilidad_cero = np.mean(utilidades == 0) * 100
        prob_utilidad_positiva = np.mean(utilidades > 0) * 100
        prob_utilidad_negativa = np.mean(utilidades < 0) * 100

        if len(utilidades_filtradas) == 0:
            return {
                'evento': nombre_evento,
                'media': 0,
                'std': 0,
                'percentil_5': 0,
                'percentil_25': 0,
                'mediana': 0,
                'percentil_75': 0,
                'percentil_95': 0,
                'minimo': 0,
                'maximo': 0,
                'prob_utilidad_cero': prob_utilidad_cero,
                'prob_utilidad_positiva': prob_utilidad_positiva,
                'prob_utilidad_negativa': prob_utilidad_negativa,
                'n_obs': 0
            }

        # Recorte de valores extremos (entre percentil 1 y 99)
        p1 = np.percentile(utilidades_filtradas, 1)
        p99 = np.percentile(utilidades_filtradas, 99)
        utilidades_trim = utilidades_filtradas[(utilidades_filtradas >= p1) & (utilidades_filtradas <= p99)]

        return {
            'evento': nombre_evento,
            'media': np.mean(utilidades_trim),
            'std': np.std(utilidades_trim),
            'percentil_5': np.percentile(utilidades_trim, 5),
            'percentil_25': np.percentile(utilidades_trim, 25),
            'mediana': np.percentile(utilidades_trim, 50),
            'percentil_75': np.percentile(utilidades_trim, 75),
            'percentil_95': np.percentile(utilidades_trim, 95),
            'minimo': np.min(utilidades_trim),
            'maximo': np.max(utilidades_trim),
            'prob_utilidad_cero': prob_utilidad_cero,
            'prob_utilidad_positiva': prob_utilidad_positiva,
            'prob_utilidad_negativa': prob_utilidad_negativa,
            'n_obs': len(utilidades_trim)
        }

    def simular_periodo(self, meses, n_simulaciones=10000):
        """Simula mÃºltiples meses"""
        resultados = {}

        for mes in meses:
            print(f"Simulando {mes}...")
            datos_mes = self.simular_mes(mes, n_simulaciones)

            # Calcular probabilidades observadas en la simulaciÃ³n
            conteo_eventos = datos_mes['conteo_eventos']
            prob_observadas = {evento: count/n_simulaciones for evento, count in conteo_eventos.items()}

            # Calcular estadÃ­sticas para cada tipo de utilidad
            stats_total = self.calcular_estadisticas(datos_mes['total'], 'Total')
            stats_devoluciones = self.calcular_estadisticas(datos_mes['devoluciones'], 'Devoluciones')
            stats_ventas = self.calcular_estadisticas(datos_mes['ventas'], 'Ventas')

            resultados[mes] = {
                'utilidades': datos_mes,
                'estadisticas': {
                    'total': stats_total,
                    'devoluciones': stats_devoluciones,
                    'ventas': stats_ventas
                },
                'conteo_eventos': conteo_eventos,
                'probabilidades_observadas': prob_observadas
            }

        return resultados

    def generar_reporte_por_tipo(self, resultados, tipo_evento):
        """Genera reporte estadístico para un tipo específico de evento con separador de miles estilo latinoamericano"""
        df_stats = pd.DataFrame()

        for mes, datos in resultados.items():
            stats = datos['estadisticas'][tipo_evento]
            df_stats[mes] = [
                stats['media'],
                stats['std'],
                stats['percentil_5'],
                stats['percentil_25'],
                stats['mediana'],
                stats['percentil_75'],
                stats['percentil_95'],
                stats['minimo'],
                stats['maximo']
            ]

        df_stats.index = ['Media', 'Desv. Estándar', 'Percentil 5%', 'Percentil 25%',
                         'Mediana', 'Percentil 75%', 'Percentil 95%', 'Mínimo', 'Máximo']

        # Formatear los números con separador de miles estilo latinoamericano
        df_stats = df_stats.applymap(lambda x: f"{x:,.0f}".replace(",", "X").replace(".", ",").replace("X", "."))

        return df_stats

    def generar_reportes_completos(self, resultados):
        """Genera todos los reportes separados"""
        reportes = {}

        for tipo in ['total', 'devoluciones', 'ventas']:
            reportes[tipo] = self.generar_reporte_por_tipo(resultados, tipo)

        return reportes

    def generar_reporte_probabilidades(self, resultados):
        """Genera reporte de probabilidades por tipo de evento"""
        df_prob = pd.DataFrame()

        for mes, datos in resultados.items():
            stats_total = datos['estadisticas']['total']
            stats_dev = datos['estadisticas']['devoluciones']
            stats_ventas = datos['estadisticas']['ventas']

            df_prob[mes] = [
                stats_total['prob_utilidad_positiva'],
                stats_total['prob_utilidad_negativa'],
                stats_total['prob_utilidad_cero'],
                stats_dev['prob_utilidad_negativa'],  # Devoluciones negativas
                100 - stats_dev['prob_utilidad_cero'],  # Devoluciones que ocurren
                stats_ventas['prob_utilidad_positiva'],  # Ventas positivas
                100 - stats_ventas['prob_utilidad_cero']  # Ventas que ocurren
            ]

        df_prob.index = [
            'Total: Utilidad Positiva (%)',
            'Total: Utilidad Negativa (%)',
            'Total: Utilidad Cero (%)',
            'Devoluciones: PÃ©rdidas (%)',
            'Devoluciones: Ocurrencia (%)',
            'Ventas: Ganancias (%)',
            'Ventas: Ocurrencia (%)'
        ]

        return df_prob

    def graficar_resultados_separados(self, resultados, mostrar_meses=4):
        """Genera grÃ¡ficos separados por tipo de evento"""
        meses_muestra = list(resultados.keys())[:mostrar_meses]

        # Crear figura con subplots para cada tipo
        fig, axes = plt.subplots(3, len(meses_muestra), figsize=(5*len(meses_muestra), 15))
        if len(meses_muestra) == 1:
            axes = axes.reshape(-1, 1)

        tipos = ['total', 'devoluciones', 'ventas']
        nombres_tipos = ['Utilidad Total', 'Utilidad Devoluciones', 'Utilidad Ventas']
        colores = ['blue', 'red', 'green']

        for i, tipo in enumerate(tipos):
            for j, mes in enumerate(meses_muestra):
                ax = axes[i, j]
                utilidades = resultados[mes]['utilidades'][tipo]
                stats = resultados[mes]['estadisticas'][tipo]

                # Filtrar valores cero para mejor visualizaciÃ³n (excepto para grÃ¡fico total)
                if tipo != 'total':
                    utilidades_filtradas = utilidades[utilidades != 0]
                    if len(utilidades_filtradas) > 0:
                        utilidades_plot = utilidades_filtradas
                    else:
                        utilidades_plot = utilidades
                else:
                    utilidades_plot = utilidades

                ax.hist(utilidades_plot, bins=50, alpha=0.7, density=True, color=colores[i])
                ax.axvline(stats['media'], color='darkred', linestyle='--',
                          label=f'Media: {stats["media"]:.0f}')
                ax.axvline(stats['mediana'], color='orange', linestyle='--',
                          label=f'Mediana: {stats["mediana"]:.0f}')

                if i == 0:  # Solo tÃ­tulo en la primera fila
                    ax.set_title(f'{mes}', fontsize=12, fontweight='bold')

                if j == 0:  # Solo ylabel en la primera columna
                    ax.set_ylabel(f'{nombres_tipos[i]}\nDensidad', fontsize=10)

                if i == len(tipos) - 1:  # Solo xlabel en la Ãºltima fila
                    ax.set_xlabel('Utilidad (COP)', fontsize=10)

                ax.legend(fontsize=8)
                ax.grid(True, alpha=0.3)

                # AÃ±adir texto con informaciÃ³n adicional
                if tipo == 'devoluciones':
                    ocurrencia = 100 - stats['prob_utilidad_cero']
                    ax.text(0.02, 0.98, f'Ocurrencia: {ocurrencia:.1f}%',
                           transform=ax.transAxes, fontsize=8, verticalalignment='top',
                           bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
                elif tipo == 'ventas':
                    ocurrencia = 100 - stats['prob_utilidad_cero']
                    ax.text(0.02, 0.98, f'Ocurrencia: {ocurrencia:.1f}%',
                           transform=ax.transAxes, fontsize=8, verticalalignment='top',
                           bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

        plt.suptitle('DistribuciÃ³n de Utilidades por Tipo de Evento', fontsize=16, y=0.98)
        plt.tight_layout()
        plt.show()

        # GrÃ¡fico comparativo de medias
        self.graficar_medias_comparativas(resultados)

    def graficar_medias_comparativas(self, resultados):
        """GrÃ¡fico comparativo de medias por tipo de evento"""
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

        meses = list(resultados.keys())
        medias_total = [resultados[mes]['estadisticas']['total']['media'] for mes in meses]
        medias_dev = [resultados[mes]['estadisticas']['devoluciones']['media'] for mes in meses]
        medias_ventas = [resultados[mes]['estadisticas']['ventas']['media'] for mes in meses]

        std_total = [resultados[mes]['estadisticas']['total']['std'] for mes in meses]
        std_dev = [resultados[mes]['estadisticas']['devoluciones']['std'] for mes in meses]
        std_ventas = [resultados[mes]['estadisticas']['ventas']['std'] for mes in meses]

        x = range(len(meses))

        # Grafico 1: Medias con barras de error
        ax1.errorbar(x, medias_total, yerr=std_total, capsize=5, marker='o',
                    linewidth=2, markersize=8, label='Total', color='blue')
        ax1.errorbar([i-0.1 for i in x], medias_dev, yerr=std_dev, capsize=5, marker='s',
                    linewidth=2, markersize=8, label='Devoluciones', color='red')
        ax1.errorbar([i+0.1 for i in x], medias_ventas, yerr=std_ventas, capsize=5, marker='^',
                    linewidth=2, markersize=8, label='Ventas', color='green')

        ax1.set_xticks(x)
        ax1.set_xticklabels(meses, rotation=45)
        ax1.set_title('Utilidad Media Mensual por Tipo de Evento')
        ax1.set_ylabel('Utilidad Media (COP)')
        ax1.legend()
        ax1.grid(True, alpha=0.3)
        ax1.axhline(y=0, color='black', linestyle='-', alpha=0.5)

        # Grafico 2: Barras apiladas (solo valores positivos y negativos por separado)
        medias_dev_pos = [max(0, x) for x in medias_dev]
        medias_dev_neg = [min(0, x) for x in medias_dev]
        medias_ventas_pos = [max(0, x) for x in medias_ventas]

        ax2.bar(x, medias_ventas_pos, label='Ventas', color='green', alpha=0.7)
        ax2.bar(x, medias_dev_neg, label='Devoluciones', color='red', alpha=0.7)

        ax2.set_xticks(x)
        ax2.set_xticklabels(meses, rotation=45)
        ax2.set_title('ContribuciÃ³n de Cada Evento a la Utilidad Total')
        ax2.set_ylabel('Utilidad Media (COP)')
        ax2.legend()
        ax2.grid(True, alpha=0.3)
        ax2.axhline(y=0, color='black', linestyle='-', alpha=0.5)

        plt.tight_layout()
        plt.show()

# Ejemplo de uso
if __name__ == "__main__":
    # Probabilidades mensuales basadas en datos histÃ³ricos reales
    probabilidades_historicas = {
            'Enero 2025': {'venta': 0.11, 'devolucion': 0.08},
            'Febrero 2025': {'venta': 0.13, 'devolucion': 0.08},
            'Marzo 2025': {'venta': 0.11, 'devolucion': 0.08},
            'Abril 2025': {'venta': 0.13, 'devolucion': 0.08},
            'Mayo 2025': {'venta': 0.08, 'devolucion': 0.08},
            'Junio 2025': {'venta': 0.06, 'devolucion': 0.08},
            'Julio 2025': {'venta': 0.06, 'devolucion': 0.08},
            'Agosto 2025': {'venta': 0.06, 'devolucion': 0.15},
            'Septiembre 2025': {'venta': 0.08, 'devolucion': 0.08},
            'Octubre 2025': {'venta': 0.05, 'devolucion': 0.08},
            'Noviembre 2025': {'venta': 0.08, 'devolucion': 0.08},
            'Diciembre 2025': {'venta': 0.06, 'devolucion': 0.08}
        }

    # Crear instancia de la simulaciÃ³n con probabilidades mensuales
    simulacion = MonteCarloSimulation(probabilidades_mensuales=probabilidades_historicas)

    # Definir meses a simular (Mayo 2025 - Diciembre 2025)
    meses = ['Mayo 2025', 'Junio 2025', 'Julio 2025', 'Agosto 2025',
             'Septiembre 2025', 'Octubre 2025', 'Noviembre 2025', 'Diciembre 2025']

    # Ejecutar simulaciÃ³n
    print("Iniciando simulaciones Monte Carlo...")
    print("Esto puede tomar unos momentos...")

    resultados = simulacion.simular_periodo(meses, n_simulaciones=10000)

    # Generar reportes separados
    reportes = simulacion.generar_reportes_completos(resultados)

    print("\n" + "="*80)
    print("REPORTE ESTADISTICO - UTILIDAD TOTAL MENSUAL")
    print("="*80)
    print(reportes['total'].round(0))

    print("\n" + "="*80)
    print("REPORTE ESTADISTICO - UTILIDADES POR DEVOLUCIONES")
    print("="*80)
    print(reportes['devoluciones'].round(0))

    print("\n" + "="*80)
    print("REPORTE ESTADISTICO - UTILIDADES POR VENTAS")
    print("="*80)
    print(reportes['ventas'].round(0))

    # Reporte de probabilidades
    reporte_probabilidades = simulacion.generar_reporte_probabilidades(resultados)
    print("\n" + "="*80)
    print("REPORTE DE PROBABILIDADES POR TIPO DE EVENTO")
    print("="*80)
    print(reporte_probabilidades.round(1))

    # AnÃ¡lisis detallado de eventos
    print("\n" + "="*80)
    print("ANALISIS DETALLADO DE EVENTOS POR MES")
    print("="*80)

    for mes in meses:
        prob_obs = resultados[mes]['probabilidades_observadas']
        stats_total = resultados[mes]['estadisticas']['total']
        stats_dev = resultados[mes]['estadisticas']['devoluciones']
        stats_ventas = resultados[mes]['estadisticas']['ventas']

        print(f"\n{mes}:")
        print(f"  Eventos observados:")
        print(f"    Solo devolucines:     {prob_obs['solo_devolucion']:.1%}")
        print(f"    Solo venta:          {prob_obs['solo_venta']:.1%}")
        print(f"    Ambos eventos:       {prob_obs['ambos_eventos']:.1%}")
        print(f"    NingÃºn evento:       {prob_obs['ningun_evento']:.1%}")

        print(f"  Utilidades promedio:")
        print(f"    Total:               {stats_total['media']:.0f} COP")
        print(f"    Devoluciones:        {stats_dev['media']:.0f} COP")
        print(f"    Ventas:              {stats_ventas['media']:.0f} COP")

        print(f"  Probabilidades de resultado:")
        print(f"    Utilidad total > 0:  {stats_total['prob_utilidad_positiva']:.1f}%")
        print(f"    Utilidad total < 0:  {stats_total['prob_utilidad_negativa']:.1f}%")
        print(f"    Utilidad total = 0:  {stats_total['prob_utilidad_cero']:.1f}%")

    # Probabilidades teÃ³ricas esperadas (ahora varÃ­an por mes)
    print("\n" + "="*80)
    print("PROBABILIDADES TEORICAS ESPERADAS POR MES")
    print("="*80)

    for mes in meses:
        prob_dev, prob_venta = simulacion.get_probabilidades_mes(mes)
        prob_solo_dev = prob_dev * (1 - prob_venta)
        prob_solo_venta = (1 - prob_dev) * prob_venta
        prob_ambos = prob_dev * prob_venta
        prob_ninguno = (1 - prob_dev) * (1 - prob_venta)

        print(f"\n{mes} (Dev: {prob_dev:.1%}, Venta: {prob_venta:.1%}):")
        print(f"  Solo devolucines:     {prob_solo_dev:.1%}")
        print(f"  Solo venta:          {prob_solo_venta:.1%}")
        print(f"  Ambos eventos:       {prob_ambos:.1%}")
        print(f"  Ningun evento:       {prob_ninguno:.1%}")

    # AnÃ¡lisis de riesgo por tipo
    print("\n" + "="*80)
    print("ANALISIS DE RIESGO POR TIPO DE EVENTO")
    print("="*80)

    for mes in meses:
        stats_total = resultados[mes]['estadisticas']['total']
        stats_dev = resultados[mes]['estadisticas']['devoluciones']
        stats_ventas = resultados[mes]['estadisticas']['ventas']

        print(f"\n{mes}:")
        print(f"  Riesgo total (prob. pÃ©rdida):     {stats_total['prob_utilidad_negativa']:.1f}%")
        print(f"  Ocurrencia devoluciones:          {100 - stats_dev['prob_utilidad_cero']:.1f}%")
        print(f"  Ocurrencia ventas:                {100 - stats_ventas['prob_utilidad_cero']:.1f}%")
        print(f"  VaR 5% total:                     {stats_total['percentil_5']:.0f} COP")

    # Generar grÃ¡ficos separados
    simulacion.graficar_resultados_separados(resultados)

    print("\Simualciones completada con reportes separados y probabilidades mensuales!")
    print(f"Se realizaron 10,000 simulaciones para cada uno de los {len(meses)} meses.")